In [1]:
import dask.dataframe as dd
import dask.bag as db
from dask.distributed import Client
import os
import requests
import json
import logging

In [2]:
DATA_DOWNLOAD_DIRECTORY = "/home/r0835817/2023-WoutRombouts-NoCsBack/ml4see/download"
DATA_SUMMARY_PATH = "/home/r0835817/2023-WoutRombouts/ml4see/data_retrieval_download.json"
NETWORK_TIMEOUT = 500

In [3]:
logging.basicConfig(filename='data_retrieval_download.log', filemode="w", level=logging.INFO, format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s')

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 125.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40093,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 125.69 GiB
Comm: tcp://127.0.0.1:33351,Total threads: 4
Dashboard: http://127.0.0.1:43195/status,Memory: 15.71 GiB
Nanny: tcp://127.0.0.1:36275,


In [5]:
if not os.path.exists(DATA_DOWNLOAD_DIRECTORY):
    logging.warning("The data download directory does not exist at {}.".format(DATA_DOWNLOAD_DIRECTORY))
    
if not os.path.exists(DATA_SUMMARY_PATH):
    logging.warning("There is no data summary file found at {}.".format(DATA_SUMMARY_PATH))

In [6]:
runs = []
with open(DATA_SUMMARY_PATH, 'r', encoding="utf-8") as file:
    runs = json.load(file)

In [7]:
def download_file(run):
    try:
        filename = os.path.join(DATA_DOWNLOAD_DIRECTORY, run["url"].split('/')[-1])
        with requests.get(run["url"], timeout=NETWORK_TIMEOUT, stream=True) as response:
            response.raise_for_status()
            with open(filename, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
        logging.info("Downloaded {}.".format(run["name"]))
    except Exception as e:
        logging.fatal("Failed to download {}!".format(run["name"]), exc_info=True)

bag = db.from_sequence(runs)
tasks = bag.map(download_file)

In [8]:
#tasks.visualize()

In [9]:
results = tasks.compute()

2023-08-27 23:18:13,332 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-08-27 23:18:13,334 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/home/r0835817/2023-WoutRombouts/miniconda3/envs/ml4see/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/r0835817/2023-WoutRombouts/miniconda3/envs/ml4see/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/r0835817/2023-WoutRombouts/miniconda3/envs/ml4see/lib/python3.11/site-packages/distributed/nanny.py", line 955, in run
    await worker.finished()
  File "/home/r0835817/2023-WoutRombouts/miniconda3/envs/ml4s

KeyboardInterrupt: 

In [10]:
client.close()